In [ ]:
import temet
from spectra_visualization_helpers import *
import matplotlib.pyplot as plt

## Compare dz from temet and my calculations

In [ ]:
gps_list = [i for i in range(50)]
base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_list]

In [ ]:
params = []
diffs = []
temet_dzs = []
my_dzs = []
for current_gp in gp_paths:
    # print(current_gp)
    sim = temet.sim(current_gp, redshift=2.0)
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(current_gp+"output/")
    dz = delta_z(25, 2, Omega0, OmegaLambda, HubbleParam)
    relative_diff = sim.dz - dz
    print(f"Difference: {relative_diff:.6f}")
    print("--------")
    params.append([Omega0, OmegaBaryon, OmegaLambda, HubbleParam])
    diffs.append(relative_diff)
    temet_dzs.append(sim.dz)
    my_dzs.append(dz)

In [ ]:
fig, ax = plt.subplots(2,2)

for j in range(4):
    row = j//2
    col = j%2
    axe = ax[row][col]
    
    cosmo_par_index = j  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
    cosmo_par = [i[cosmo_par_index] for i in params]
    label_dir = {0: "Omega0", 1: "OmegaBaryon", 2: "OmegaLambda", 3: "HubbleParam"}

    axe.scatter(cosmo_par, diffs, marker="x", label="Differences")

    axe.set_ylabel("Difference in z")
    axe.set_xlabel(f"{label_dir[j]}")

plt.tight_layout()
plt.show()

In [ ]:
def a_lambda_only(t, Omega_Lambda, h):
    H_0 = 100 * h * 1/(3.086*1e19)  # in 1/s
    return np.exp(np.sqrt(Omega_Lambda)*H_0*t)


def t_lambda_only(a, Omega_Lambda, h):
    H_0 = 100 * h * 1/(3.086*1e19)  # in 1/s
    return np.log(a)/(np.sqrt(Omega_Lambda)*H_0)


def delta_z_lambda_only(d_c, z_start, Omega_Lambda, h):
    """
    d_c (float)     : distance in cMpc/h
    a_start (float) : scale factor of box
    """
    a_start = 1/(z_start + 1)
    c = 299792.458  # in km/s
    t_tr = (d_c/h)*a_start/c *10**6 *3.086*10**13  # in s
    t_start = t_lambda_only(a_start, Omega_Lambda, h)

    d_z = 1/a_start - 1/a_lambda_only(t_start+t_tr, Omega_Lambda, h)
    return d_z


In [ ]:
diff_lambda_only = []
my_dzs_lambda_only = []
for i, current_gp in enumerate(gp_paths):
    # print(current_gp)
    dz_lambda_only = delta_z_lambda_only(25, 2, OmegaLambda, HubbleParam)
    relative_diff = temet_dzs[i] - dz_lambda_only
    print(f"Difference: {relative_diff:.6f}")
    print("--------")
    diff_lambda_only.append(relative_diff)
    my_dzs_lambda_only.append(dz)

In [ ]:
fig, ax = plt.subplots(2,2)

for j in range(4):
    row = j//2
    col = j%2
    axe = ax[row][col]
    
    cosmo_par_index = j  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
    cosmo_par = [i[cosmo_par_index] for i in params]
    label_dir = {0: "Omega0", 1: "OmegaBaryon", 2: "OmegaLambda", 3: "HubbleParam"}

    axe.scatter(cosmo_par, my_dzs, marker="o", label="my calcs", alpha=1, s=4)
    axe.scatter(cosmo_par, temet_dzs, marker="o", label="temet", alpha=1, s=4)

    axe.set_ylabel("z")
    axe.set_xlabel(f"{label_dir[j]}")
    axe.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.scatter(diffs, my_dzs, label="my calcs")
plt.scatter(diffs, temet_dzs, label="temet")
plt.ylabel("z")
plt.xlabel("difference in z between methods")
plt.legend()
plt.show()